## Notebook For Eclipse Software Deployment ##
<i>Written by David Story </i>

___
### Description
This notebook was written to be an easy-to-use plateform for deploying and implementing software to AWS cloud computing services and creating datasets for the Eclipse Megamovie project, as well as related computationally heavy research projects.

This notebook will automate a number of tasks for preparing your AWS EC2 instances and deploying software to those instances. 

This notebook will:
<ul style=squares>
    <li>Create a specified amount of instances</li>
    <li>Send datasets to those instances</li>
    <li>Send processing jobs and parameters to instances</li>
    <li>Run jobs</li>
    <li>Monitor jobs</li>
    <li>Retrieve and store results</li>
</ul>

___
### Information on Provided Files ###
This repository contains the following folders:
<ul style=squares>
    <li>examples</li>
    <li>keys</li>
    <li>documentation</li>
    <li>images</li>
    <li>movies</li>
    <li>logs</li>
    <li>figures</li>
    <li>results</li>
</ul>


___
#### examples: ####
This folder provides example files that will allow you to understand how files are being fed to this notebook, and how they are formated, to help you understand how to generate your files to allow this notebook to run smoothly.

#### keys: ####
This folder should hold your ssh or .pem keys that you generated for your EC2 instance

#### documentation: ####
This should should hold the .csv that describes your EC2 instance names and attributes.

#### images: ####
This should hold any images generated.

#### movies: ####
This folder should hold any movies generated.

#### servers: ####
This folder should hold any logs generated relating to server performance, states, and program performance and states.

#### figures: ####
This folder should hold any figures generated.

#### results: ####
This folder should hold any results generated.

___
### Dependencies
This notebook requires the following dependenices
<ul style=squares>
    <li>boto3</li>
    <li>botocore</li>
    <li>jmespath</li>
    <li>s3transfer</li>
</ul>

___
### Setting up AWS Config

We will be using the Boto3 API to interface Python with AWS, before we can do that we need to instance the AWS CLI from here: https://aws.amazon.com/cli/

After you have installed the CLI, you must configure your AWS credentials on the machine that you will run this code on, the configuration process can be found here: https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html#configuration

<b>Ensure that in the configure step you set your region to the region that your EC2 and S3 instances and buckets are at, else you will not be able to access them with the API</b> 

___

### Viewing your current EC2 Instances###

Using the Boto3 library, we are able to acquire information about our instances that are in AWS using this Python API, below we are importing the libraries we will need:

In [ ]:
import sys
import boto3
from botocore.exceptions import ClientError

Now that the libraries are acquired, we will use the ec2 resource to see what instances that are stopped are on our console:

In [ ]:
# variable for ec2 resource
ec2 = boto3.resource('ec2')

# printing what running instances we have up
print("EC2 Instances Running:")

# filtering for instances by name and if running
running_instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 'Values': ['running']}])

# printing the names of running instances and their machine type
for instance in running_instances:
    print(instance.id, instance.instance_type)

# printing what stopped instances we have
print("\nEC2 Instances Stopped:")

# filtering for instances by name and if stopped
stopped_instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 'Values': ['stopped']}])

# printing the names of running instances and their machine type
for instance in stopped_instances:
    print(instance.id, instance.instance_type)

# for other cases other than running or stopped
other_instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 'Values': ['pending', 'shutting-down',
                                                    'terminated','stopping']}])

print("\nEC2 Instances Other:")
for instance in other_instances:
    print(instance.id, instance.instance_type)
    
print("\nDone Searching")

### Setting up the instances needed ###

We will now be preparing the instances that we want to process the images with.

<b> Warning: in the following process we will be stopping all of the current instances and creating a defined amount of instances that we will use for our image processing </b>

You will be asked to confirm that you want to begin the process of pausing your instances and starting the server process.

In [ ]:
# Waits for user to authorize stop of their instances
go = False
while(go == False):
    user = input("Authorize to stop and store current instances ([Y]/n): ")
    if (user == "Y"):
        go = True
    elif (user == 'n'):
        print("Entered No: Exiting Program")
        sys.exit()
    else:
        print("Invalid Input:", user)

# Filters a list of all instances before we create new ones
old_instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 
              'Values': ['running', 'stopped',
                        'pending','shutting-down'
                         ,'terminated','stopping']
             }])

# Stores ID of old instances and then stops all instances
print("Stopping:")
stopIds = []
for instance in old_instances:
    instance.stop(instance.id)
    stopIds.append(instance.id)
print(stopIds)    

### Creating new instances ###

Now that the above instances are stopped, we want to create the number of instances that we want to use. 

Input the number of instances you want to use below:

In [ ]:
number_of_instances = abs(int(input("Number of instances to use:")))
print(number_of_instances, "EC2 instances will be created.")

We will now create a key pair that will use to SSH into our servers. We will share the same key with every server for convenience.

In [ ]:
# Creating key pair for Eclipse Megamovie usages
try:
    ec2.create_key_pair(KeyName='EMkey')
except:
    print("Key already created or invalid.")

Below are the parameters for defining the instances, they will be preset to start a set of computationationally optimized ubuntu servers

In [ ]:
# Instance parameters
image_id = ''
min_count = 1
max_count = number_of_instances
key_name = ''
instance_type = ''
security_group = ''

# EBS storage parameters
availability = ''
size = ''
snapshot_id = ''
volume_type = ''

In [ ]:
volumes = ec2.create_volume(AvailabilityZone=availability, Size=size,
                              SnapshotId=snapshot_id, VolumeType=volume_type)